In [1]:
import sys
sys.path.insert(0, 'C:/Users/Lenovo/Desktop/exam_eti/containerized_tool/data_analysis_tool/src')

## Creating Spark Session

In [2]:
import os
SCALA_VERSION = '2.12'
SPARK_VERSION = '3.1.2'
import findspark
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_{SCALA_VERSION}:{SPARK_VERSION} pyspark-shell'
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, IntegerType, StringType, TimestampType, StructField
from pyspark.context import SparkContext
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [3]:
import mysql_analyzer
import mysql.connector
import pandas as pd
from mysql.connector.errors import Error
import dat
from faker import Faker
import random
import json
from json import dumps
from json import loads
import time
from datetime import datetime
from kafka import KafkaProducer, TopicPartition, KafkaConsumer
import xlsxwriter
import openpyxl

working


## Creating Spark session with gathering stream package and some configs around

In [4]:
spark = SparkSession.builder\
        .master('local[*]')\
        .config("spark.streaming.stopGracefullyOnShutdown", True) \
        .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0') \
        .config("spark.sql.shuffle.partitions", 4) \
        .appName("myAppName")\
        .getOrCreate()

## Initiating mysql_analyzer class

In [5]:
mysql_works = mysql_analyzer.mysql_profiler('localhost',os.environ['MYSQLSERVER_USER'],
                os.environ['MYSQLSERVER_PASS'],'sakila')

## Created CDC table for customer table

In [6]:
# mysql_works.multiple_dataset_apply_mysql_insert("CREATE TABLE \
# sakila.users_change_events (log_id BIGINT AUTO_INCREMENT,\
#   event_type      TEXT,\
#   event_timestamp TIMESTAMP,\
#   user_id         INT,\
#   user_name       TEXT,\
#   user_email      TEXT,\
#   PRIMARY KEY (log_id))")



## Created Trigger that insert CDC transactions into CDC table created on previous step

In [7]:
# mysql_works.multiple_dataset_apply_mysql_insert("CREATE TRIGGER sakila.user_insert_capture AFTER INSERT ON sakila.customer FOR EACH ROW \
#   BEGIN INSERT INTO sakila.users_change_events \
#   (event_type, \
#    event_timestamp, \
#    user_id, \
#    user_name, \
#    user_email) \
#  VALUES ( \
#    'INSERT', \
#    now(), \
#    user_id, \
#    user_name, \
#    user_email); \
#   END;") 


## Created lastest cdc timestamp holding table 
### We are reading cdc last timestamp from previous data write action

In [8]:
# mysql_works.multiple_dataset_apply_mysql_insert("CREATE TABLE \
# sakila.latest_cdc_timestamp (log_id BIGINT AUTO_INCREMENT,\
#   event_type      TEXT,\
#   event_timestamp TIMESTAMP,\
#   PRIMARY KEY (log_id))")

In [9]:
# mysql_works.multiple_dataset_apply_mysql_insert(f"INSERT INTO sakila.latest_cdc_timestamp \
#                                                 (event_type,event_timestamp) \
#                                                 VALUES ('{mysql_works.multiple_dataset_apply_mysql_query \
#                                                  ('SELECT event_type FROM users_change_events ORDER BY event_timestamp DESC LIMIT 1')[0][0]}','{mysql_works.multiple_dataset_apply_mysql_query('SELECT event_timestamp FROM users_change_events ORDER BY event_timestamp DESC LIMIT 1')[0][0]}')")

## Kafka script to write data

### Producer Kafka

In [10]:
# Inserting into latest_cdc_timestamp table, latest CDC timestamp from CDC table to be able to detect if any new CDC occurred
mysql_works.multiple_dataset_apply_mysql_insert(f"INSERT INTO sakila.latest_cdc_timestamp \
                                                (event_type,event_timestamp) \
                                                VALUES ('{mysql_works.multiple_dataset_apply_mysql_query('SELECT event_type FROM users_change_events ORDER BY event_timestamp DESC LIMIT 1')[0][0]}','{mysql_works.multiple_dataset_apply_mysql_query('SELECT event_timestamp FROM users_change_events ORDER BY event_timestamp DESC LIMIT 1')[0][0]}')")
# # Creating fake records to create data flow to MYSQL db
mysql_works.fake_record_creator_sakila()

# Getting CDC timestamps from related table to use on below if statement
latest_saved_cdc_log = mysql_works.multiple_dataset_apply_mysql_query('SELECT max(event_timestamp) \
                                                                      FROM latest_cdc_timestamp')[0][0]

latest_real_cdc_log = mysql_works.multiple_dataset_apply_mysql_query('SELECT max(event_timestamp) \
                                                                     FROM users_change_events')[0][0]

# If CDC occurred instantiating KafkaProducre class and saving data into Kafka topic
if latest_saved_cdc_log < latest_real_cdc_log:
    producer = KafkaProducer(
        bootstrap_servers='settled-terrapin-12518-eu2-kafka.upstash.io:9092',
        sasl_mechanism='SCRAM-SHA-256',
        security_protocol='SASL_SSL',
        sasl_plain_username='c2V0dGxlZC10ZXJyYXBpbi0xMjUxOCTBb5AEffUiTulATzsbFtDRxbvhkO0Wsnc',
        sasl_plain_password='N2E2ZGVjY2UtZDY4YS00MjM4LTk5NTktMjU1OTRiZWQ4Y2Ix',
        value_serializer = lambda m : dumps(m, default=str).encode("utf-8")
        # api_version_auto_timeout_ms=100000,    
    )

    for record in mysql_works.multiple_dataset_apply_mysql_query(f'SELECT * FROM customer WHERE last_update > "{latest_saved_cdc_log}"'):
        data_dict = {"customer_id" : record[0],"store_id" : record[1],"first_name" : record[2] \
                     ,"first_name" : record[3],"email" : record[4],"address_id" : record[5] \
                      ,  "last_update" : record[8]}
        producer.send("mysql_write",data_dict) 
    producer.close()

1179
1180
1181
1182
1183
1184
1185
1186
1187
1188
1189
1190
1191
1192
1193
1194
1195
1196
1197
1198


### Consumer Kafka saved data in local xlsx

In [11]:
topic = 'mysql_write'
tp = TopicPartition(topic,0)

consumer = KafkaConsumer(
    bootstrap_servers='settled-terrapin-12518-eu2-kafka.upstash.io:9092',
    sasl_mechanism='SCRAM-SHA-256',
    security_protocol='SASL_SSL',
    sasl_plain_username='c2V0dGxlZC10ZXJyYXBpbi0xMjUxOCTBb5AEffUiTulATzsbFtDRxbvhkO0Wsnc',
    sasl_plain_password='N2E2ZGVjY2UtZDY4YS00MjM4LTk5NTktMjU1OTRiZWQ4Y2Ix',
    # group_id='bacak',
    auto_offset_reset='earliest',
    # max_poll_interval_ms=30,
    value_deserializer=lambda x: loads(x.decode('utf-8'))
)

# Configs to get latest offset
consumer.assign([tp])

# obtain the last offset value
consumer.seek_to_end(tp)
lastOffset = consumer.position(tp)

# Configs to get latest offset
consumer.seek_to_beginning(tp)   

# Saved topic messages into dataframe
df = pd.DataFrame()

# Stopped reading at latest offset
for message in consumer:
    if message.offset == lastOffset - 1:
        break
    else:
        message = message.value;
        # print(message)
        # print('{}'.format(message))
        df_raw = pd.json_normalize(message, max_level=0)
        df = pd.concat([df, df_raw])
        # print(df_ult)
        # dataframe_dict[]

# 

df.to_csv('stream_df.csv', index=False) 
# writer = pd.ExcelWriter("stream_df.xlsx", engine="xlsxwriter")
# df.to_excel(writer, sheet_name = "df")
# writer.close()


consumer.close()

### Read saved xlsx as spark stream source

In [12]:
## Source Schema
df_schema = StructType([
    StructField("customer_id", StringType(), True),
    StructField("store_id", StringType(), True),
    StructField("first_name", StringType(), True),
    StructField("email", StringType(), True),
    StructField("address_id", StringType(), True),
    StructField("latest_update", TimestampType(), True)
])

In [13]:
csvDF = spark \
    .readStream \
    .option("sep", ",") \
    .schema(df_schema) \
    .csv("C:/Users/Lenovo/Desktop/exam_eti/containerized_tool/data_analysis_tool/src/playground_notebooks")

In [14]:
csvDF.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- store_id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- address_id: string (nullable = true)
 |-- latest_update: timestamp (nullable = true)



In [15]:
# store_count = csvDF.groupBy("store_id").count()

In [16]:
windowedCounts = csvDF \
    .withWatermark("latest_update", "1 minutes") \
    .groupBy(F.window(csvDF.latest_update, "30 seconds"),
        csvDF.store_id) \
    .count()

In [17]:
windowedCounts.printSchema()

root
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- store_id: string (nullable = true)
 |-- count: long (nullable = false)



In [18]:
# expression = ["cast(window as string) as window", "cast('store_id' as string) as store_id", "cast('count' as string) as count"]
# windowedCounts=windowedCounts.selectExpr(expression)
# expression = ["cast(window as binary) as window", "cast('store_id' as binary) as store_id", "cast('count' as binary) as count"]
# windowedCounts=windowedCounts.selectExpr(expression)


In [19]:
windowedCounts.printSchema()

root
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- store_id: string (nullable = true)
 |-- count: long (nullable = false)



In [20]:
# windowedCounts.writeStream \
#     .outputMode("complete") \
#     .format("json").option("path", "C:/Users/Lenovo/Desktop/exam_eti/containerized_tool/data_analysis_tool/src/playground_notebooks/csv_sink") \
#     .option("checkpointLocation", "C:/Users/Lenovo/Desktop/exam_eti/containerized_tool/data_analysis_tool/src/playground_notebooks/csv_sink") \
#     .start()

# windowedCounts.writeStream\
#     .outputMode("complete") \
#     .format("console").start()

# danyal = windowedCounts.writeStream.format("kafka").outputMode("complete")  \
#     .option("kafka.bootstrap.servers",'settled-terrapin-12518-eu2-kafka.upstash.io:9092')\
#     .option("kafka.sasl.mechanism", "SCRAM-SHA-256")\
#     .option("kafka.security.protocol", "SASL_SSL")\
#     .option("kafka.sasl.jaas.config","""org.apache.kafka.common.security.plain.PlainLoginModule required username="c2V0dGxlZC10ZXJyYXBpbi0xMjUxOCSqaSFgt-fI-8JyIV50sk_wCOG7dRr8LsY" password="Y2FhZGE3ZWQtYzQxOC00ZTdiLWJlZjUtOGRhMjJjN2YwZjU1";""")\
#     .option("topic", "aggregated_store_id") \
#     .option("checkpointLocation", "C:/Users/Lenovo/Desktop/exam_eti/containerized_tool/data_analysis_tool/src/playground_notebooks/csv_sink") \
#     .start()

# danyal.awaitTermination()

In [21]:
windowedCounts \
    .writeStream \
    .queryName("didik") \
    .outputMode("complete") \
    .format("memory") \
    .start()

In [25]:
spark.sql("select * from didik").show() 

+--------------------+--------+-----+
|              window|store_id|count|
+--------------------+--------+-----+
|{2024-06-28 15:39...|       1|    4|
|{2024-06-28 15:14...|       1|    4|
|{2024-06-28 15:11...|       1|    5|
|{2024-06-28 15:15...|       1|    4|
|{2024-06-28 15:38...|       1|    4|
|{2024-06-28 15:39...|       1|    4|
|{2024-06-28 15:19...|       1|    5|
|{2024-06-28 16:10...|       2|    2|
|{2024-06-28 15:43...|       1|    5|
|{2024-06-28 14:34...|       1|    2|
|{2024-06-28 15:43...|       1|    4|
|{2024-06-28 15:20...|       1|    4|
|{2024-06-28 15:19...|       1|    5|
|{2024-06-28 14:06...|       1|    4|
|{2024-06-28 15:36...|       1|    4|
|{2024-06-28 16:10...|       1|    2|
|{2024-06-28 15:34...|       1|    3|
|{2024-06-28 15:14...|       1|    4|
|{2024-06-28 15:37...|       1|    5|
|{2024-06-28 15:13...|       1|    3|
+--------------------+--------+-----+
only showing top 20 rows



## Previous works on below chunks

In [ ]:
# mysql_works.multiple_dataset_apply_containing_cols_mysql()

In [ ]:
# mysql_works.find_value_mysql('Hillyer')

In [ ]:
# mysql_works.multiple_dataset_apply_mysql_query('SELECT * FROM criket_table')

In [ ]:
# mysql_works.multiple_dataset_apply_mysql_query('SELECT team, COUNT(team) as game_played,\
#                                           CAST(SUM(CASE WHEN team=WonBy THEN 1 ELSE 0 END)AS SIGNED) AS game_won FROM\
#                                          (SELECT TeamA AS team, WonBy from criket_table\
#                                          UNION ALL\
#                                          SELECT TeamB AS team, WonBy from criket_table) AS sub\
#                                          GROUP BY team\
#                                          ORDER BY game_won'
#                                          )